In [5]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import csv
import pickle

In [6]:
with open('../data/ip_2_mac_dict.pickle', 'r') as f:
    ip_2_mac_dict = pickle.load(f)
    
with open('../data/host_data_dict.pickle', 'r') as f:
    host_data_dict = pickle.load(f)

IOError: [Errno 2] No such file or directory: '../data/ip_2_mac_dict.pickle'

In [ ]:
import glob
dns_fnames = glob.glob("data/tshark_20170207_20170212/*.csv")

for dns_fname in dns_fnames:
    print "Processing file", dns_fname
    dns_data = []
    out_str = ''
    with open(dns_fname, 'r') as f:
        for line in f:
            tmp = line.split('|')
            if len(tmp) > 2:
                tmp[-1] = tmp[-1].split('\n')[0]
                ip_addr_str = tmp[2]
                if ip_addr_str in host_data_dict:
                    tmp += list(host_data_dict[ip_addr_str])
                else:
                     tmp += ['','','', '','','']     
            dns_data.append(tuple(tmp))
            out_str += ','.join([str(x) for x in tmp])+'\n'
        out_str = out_str[:-1]
    dump_fname = 'data/tshark_20170207_20170212_consolidated/'+dns_fname.split('.csv')[0].split('data/tshark_20170207_20170212/')[1]+'.csv'
    print "Dumping file:", dump_fname
    with open(dump_fname, 'w') as f:
        f.write(out_str)
        